<a href="https://colab.research.google.com/github/Aryalbro/Shreveport_weather_predictor/blob/main/Weather_predicting__model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
#weather_data = pd.read_csv('/content/drive/MyDrive/3856641.csv')
# importing all the necessary dependencies

In [ ]:

from google.colab import drive
drive.mount('/content/drive')
path ="/content/drive/MyDrive/3856641.csv"
#Loading & reading files using pandas

df = pd.read_csv("/content/drive/MyDrive/3856641.csv")

In [ ]:
# Convert the 'DATE' column to datetime
df['DATE'] = pd.to_datetime(df['DATE'])

# Set the 'DATE' column as the index
df.set_index('DATE', inplace=True)

# Display the first few rows of the updated DataFrame
print(df.head())


In [ ]:
print(df.columns)

In [ ]:
print(df.head())

In [ ]:
# Calculate the total number of null values in the entire DataFrame
total_nulls = df.isnull().sum().sum()

print(f"Total number of null values: {total_nulls}")


In [ ]:
#Calculates the total null values in each column
nulls_per_column = df.isnull().sum()
print(nulls_per_column)


In [ ]:
# renaming df to weather

weather = df
#weather.loc["2002-01-01":"2005-01-01"]


In [ ]:
#Cleaning up data for ML
weather.apply(pd.isnull).sum()/weather.shape[0]

In [ ]:
# copying the core variables into a new table/columns.
core_weather = weather[["PRCP", "SNOW", "SNWD", "TMAX", "TMIN"]].copy()

In [ ]:
# renaming
core_weather.columns = ["precip", "snow", "snow_depth", "max_temp", "min_temp"]
core_weather

In [ ]:
#filling the null values

In [ ]:
#checking the amt. of null values present
core_weather.apply(pd.isnull).sum()/core_weather.shape[0]

In [ ]:
# As snow doesnot have any historical data
core_weather["snow"].value_counts()
del core_weather["snow"]

In [ ]:
#same w/ snowdepth
core_weather["snow_depth"].value_counts()
del core_weather["snow_depth"]

In [ ]:
# looking for null value of preciptation
core_weather[pd.isnull(core_weather["precip"])]

In [ ]:
core_weather.loc["2018-01-10":"2018-01-30"]

In [ ]:
core_weather["precip"].value_counts()

In [ ]:
#Filling the precipitation w/ zero
core_weather["precip"] = core_weather["precip"].fillna(0)

In [ ]:
core_weather[pd.isnull(core_weather["max_temp"])]

In [ ]:
#using forward fill for the temperature

In [ ]:
# filling the temperature that are missing
core_weather = core_weather.fillna(method="ffill")

In [ ]:
core_weather[pd.isnull(core_weather["max_temp"])]

In [ ]:
core_weather[pd.isnull(core_weather["min_temp"])]

In [ ]:
#checking the null values(if any)
core_weather.apply(pd.isnull).sum()/core_weather.shape[0]

In [ ]:
#ensuring the prediction values are numericals
core_weather.dtypes

In [ ]:
# regarding the index, converting these into date/time index for easier value
#manipulation
core_weather.index

In [ ]:
# converting to date time index
core_weather.index = pd.to_datetime(core_weather.index)
core_weather.index

In [ ]:
#ensuring there are no unavailable datas
core_weather.apply(lambda x: (x==9999).sum())

In [ ]:
# Analyzibng the data

In [ ]:
# analysis in the plot form
core_weather[["max_temp","min_temp"]].plot()

In [ ]:
core_weather.index.year.value_counts().sort_index()

In [ ]:
#plotting the preciptation
core_weather["precip"].plot()

In [ ]:
# seeing how much it rained each yrs
core_weather.groupby(core_weather.index.year).sum()["precip"]

In [ ]:
# training the models:

In [ ]:
core_weather

In [ ]:
core_weather = core_weather.iloc[:-1,:].copy()

In [ ]:
# importing ML model
from sklearn.linear_model import Ridge # to prevent overfitting
reg = Ridge(alpha=.1)

In [ ]:
predictors = ["precip", "max_temp", "min_temp"]

In [ ]:
core_weather["target"] = core_weather.shift(-1)["max_temp"]

In [ ]:

train = core_weather.loc[:"2019-04-01"]
test = core_weather.loc["2019-05-01":]

In [ ]:
# fitting the model
reg.fit(train[predictors], train["target"])

In [ ]:
predictions = reg.predict(test[predictors])

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
core_weather

In [ ]:
#checking the null values(if any)
core_weather.apply(pd.isnull).sum()/core_weather.shape[0]

In [ ]:
core_weather = core_weather.iloc[:-1,:].copy()
#mean_absolute_error(test["target"], predictions)

In [ ]:
core_weather

In [ ]:
mean_absolute_error(test["target"], predictions)

In [ ]:
combined = pd.concat([test["target"], pd.Series(predictions, index=test.index)], axis=1)

In [ ]:
combined.columns = ["actual", "predictions"]

In [ ]:
combined.plot()

In [ ]:
reg.coef_

In [ ]:
#if rains, tempr, low

In [ ]:
def create_predictions(predictors, core_weather, reg):
    train = core_weather.loc[:"2019-04-01"]
    test = core_weather.loc["2019-05-01":]

    reg.fit(train[predictors], train["target"])
    predictions = reg.predict(test[predictors])

    error = mean_squared_error(test["target"], predictions)

    combined = pd.concat([test["target"], pd.Series(predictions, index=test.index)], axis=1)
    combined.columns = ["actual", "predictions"]
    return error, combined

In [ ]:
#rolling mean, calculates the mean every 30 days
core_weather["month_max"] = core_weather["max_temp"].rolling(30).mean()

In [ ]:
core_weather

In [ ]:
core_weather["month_day_max"] = core_weather["month_max"] / core_weather["max_temp"]

In [ ]:
core_weather["max_min"] = core_weather["max_temp"] / core_weather["min_temp"]

In [ ]:
#adding&combining all predictors
predictors = ["precip", "max_temp", "min_temp", "month_max", "month_day_max", "max_min"]

In [ ]:
# removing the newly created NaN errors
core_weather = core_weather.iloc[30:,:].copy()

In [ ]:
error, combined = create_predictions(predictors, core_weather, reg)

In [ ]:
error

In [ ]:
combined.plot()

In [ ]:
core_weather["monthly_avg"] = core_weather["max_temp"].groupby(core_weather.index.month).transform(lambda x: x.expanding(1).mean())
core_weather["day_of_year_avg"] = core_weather["max_temp"].groupby(core_weather.index.day_of_year).transform(lambda x: x.expanding(1).mean())

In [ ]:
core_weather

In [ ]:
predictors = ["precip", "max_temp", "min_temp", "month_max", "max_min", "day_of_year_avg", "monthly_avg"]

In [ ]:
error, combined = create_predictions(predictors, core_weather, reg)
error

In [ ]:
reg.coef_

In [ ]:
# correlations to the target
core_weather.corr()["target"]

In [ ]:
#trying to find the differece
combined["diff"] = (combined["actual"] - combined["predictions"]).abs()

In [ ]:
combined.sort_values("diff", ascending=False).head(10)